In [1]:
import awswrangler as wr
import mlflow

%env AWS_ACCESS_KEY_ID=minio   
%env AWS_SECRET_ACCESS_KEY=minio123 
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
%env AWS_ENDPOINT_URL_S3=http://localhost:9000

env: AWS_ACCESS_KEY_ID=minio
env: AWS_SECRET_ACCESS_KEY=minio123
env: MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
env: AWS_ENDPOINT_URL_S3=http://localhost:9000


# Búsqueda de mejor modelo e hiperparámetros

Dado nuestro dataset, el cual ya pasó por el proceso de ETL y se encuentra en nuestro S3 bucket, vamos a realizar una búsqueda de cual seria el mejor modelo y que hiperparametros usar.

La búsqueda de hiperparametros la haremos usando Optuna y el tracking será realizado mediante MLFlow.

OBS: Para la confección de esta notebook, nos basamos en el tutorial de [MLFlow](https://mlflow.org/docs/latest/traditional-ml/hyperparameter-tuning-with-child-runs/notebooks/index.html).

In [2]:
mlflow_server = "http://localhost:5000"

mlflow.set_tracking_uri(mlflow_server)

In [3]:
# Cargamos los datos
X_train =  wr.s3.read_csv("s3://data/train/X_train.csv")
y_train =  wr.s3.read_csv("s3://data/train/y_train.csv")

X_test =  wr.s3.read_csv("s3://data/test/X_test.csv")
y_test =  wr.s3.read_csv("s3://data/test/y_test.csv")

## Investigamos la correlación de features con la variable objetivo

Antes de profundizar en el proceso de construcción de modelo, es esencial comprender las relaciones entre nuestras features  y la variable objetivo. Por lo que vamos a realizar un gráfico que indica el coeficiente de correlación de cada feature en relación con la variable objetivo. Esto nos sirve para:

- Evitar data leakage: Debemos asegurarnos de que ninguna característica se correlacione perfectamente con el objetivo (un coeficiente de correlación de aproximadamente 1.0). Si existe tal correlación, es una señal de que nuestro conjunto de datos podría estar "filtrando" información sobre el objetivo. 

- Garantizar relaciones significativas: Idealmente, nuestras características deberían tener algún grado de correlación con el objetivo. Inclusive si estamos trabajando con un problema de clasificación, aunque los resultados no son tan importantes como en un caso de regresión.

- Auditoría y trazabilidad: Loggear esta visualización de correlación con nuestra ejecución principal de MLflow garantiza la trazabilidad. Proporciona una instantánea de las características de los datos en el momento del entrenamiento del modelo, lo cual es invaluable para propósitos de auditoría y replicabilidad.

In [4]:
from plots import plot_correlation_with_target, plot_information_gain_with_target

In [5]:
# Dado que estamos usando como tracking a MLFlow, mostrar los gráficos aquí no tiene sentido.
correlation_plot = plot_correlation_with_target(X_train, y_train, target_col="cantidad_robos_log")
information_gain_plot = plot_information_gain_with_target(X_train, y_train, target_col="cantidad_robos_log")

## Arrancamos a experimentar

In [6]:
import datetime
import optuna

from mlflow.models import infer_signature
from mlflow_aux import get_or_create_experiment

from optuna_aux import champion_callback, objective

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor

from sklearn.metrics import mean_absolute_error

# Optuna es un poco verboso, dejamos que solo nos muestre logs de errores
optuna.logging.set_verbosity(optuna.logging.ERROR)

c:\Users\masso\OneDrive\Escritorio\amq2-tp-final\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Antes de poder realizar experimentos, vamos a crear el experimento en MLFLow, pero para evitar desorden, vamos a usar una función que se fije primero si el experimento existe, si esto es así, devuelve su ID.

Además creamos el nombre del run padre con el que vamos a ir registrando las ejecuciones.

In [7]:
# Creemos el experimento
experiment_id = get_or_create_experiment("hyperparam_tuning")
print(experiment_id)

run_name_parent = "best_hyperparam_"  + datetime.datetime.today().strftime('%Y/%m/%d-%H:%M:%S"')

3


Ya con todo seteado, vamos a ejecutar la optimización usando Optuna, el cual realiza una búsqueda Bayesiana, la cual es más eficiente que una búsqueda de grilla tradicional. La desventaja es que es más difícil de paralelizar.

In [8]:
with mlflow.start_run(experiment_id=experiment_id, run_name=run_name_parent, nested=True):
    # Inicializamos el estudio de Optuna
    study = optuna.create_study(direction="minimize")

    # Ejecutamos los trials de optimización de hiperparametros. Cada uno de estos trials se ejecuta con un run separado, pero 
    # está anidado al run padre.
    # Notar la adición del `champion_callback` para controlar qué mensajes mostramos
    # Para entender mejor esto ver la documentación de objective y champion_callback en optuna_aux
    study.optimize(lambda trial: objective(trial, X_train, y_train, experiment_id), n_trials=100, callbacks=[champion_callback])

    # Una vez que terminamos la búsqueda, guardamos los mejores parámetros en el run padre.
    mlflow.log_params(study.best_params)
    mlflow.log_metric("train_mae", study.best_value)

    mlflow.set_tags(
        tags={
            "project": "Crime Prediction",
            "optimizer_engine": "optuna",
            "model_family": "sklearn",
            "feature_set_version": 1,
        }
    )

    # Una vez que terminamos la búsqueda, nos quedamos con el mejor modelo y lo entrenamos
    if study.best_params["model"] == 'LinearRegression':
        model = LinearRegression()
    elif study.best_params["model"] == 'Ridge':
        model = Ridge(alpha=study.best_params["ridge_alpha"]) 
    elif study.best_params["model"] == 'SVR':
        model = SVR(C=study.best_params["svr_C"], gamma=study.best_params["svr_gamma"], kernel=study.best_params["svr_kernel"])
    elif study.best_params["model"] == 'DecisionTree':
        model = DecisionTreeRegressor(max_depth=study.best_params["dt_max_depth"])
    elif study.best_params["model"] == 'RandomForest':
        model = RandomForestRegressor(n_estimators=study.best_params["rf_n_estimators"], max_depth=study.best_params["rf_max_depth"])
    elif study.best_params["model"] == 'AdaBoost':
        model = AdaBoostRegressor(n_estimators=study.best_params["ab_n_estimators"], learning_rate=study.best_params["ab_learning_rate"])
    elif study.best_params["model"] == 'XGBoost':
        model = XGBRegressor(n_estimators=study.best_params["xgb_n_estimators"], 
                             learning_rate=study.best_params["xgb_learning_rate"], 
                             max_depth=study.best_params["xgb_max_depth"], 
                             use_label_encoder=False, eval_metric='logloss')
    
    model = model.fit(X_train, y_train)

    # Testeamos el modelo y logueamos el resultado
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mlflow.log_metric("test_mae", mae)

    # Logueamos los artefactos de las gráficas de correlación y de information_gain
    mlflow.log_figure(figure=correlation_plot, artifact_file="correlation_plot.png")
    mlflow.log_figure(figure=information_gain_plot, artifact_file="information_gain_plot.png")

    # Guardamos el artefacto del modelo
    artifact_path = "model"

    signature = infer_signature(X_train, model.predict(X_train))

    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path=artifact_path,
        signature=signature,
        serialization_format='cloudpickle',
        registered_model_name="crime_prediction_model_dev",
        metadata={"model_data_version": 1}
    )

    # Obtenemos la ubicación del modelo guardado en MLFlow
    model_uri = mlflow.get_artifact_uri(artifact_path)


2024/08/15 19:54:43 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 0 at: http://localhost:5000/#/experiments/3/runs/7d5dfef6495343c7ba6460859d798fe0.
2024/08/15 19:54:43 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/3.


Initial trial 0 achieved value: 0.46019596700628923


2024/08/15 19:54:46 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 1 at: http://localhost:5000/#/experiments/3/runs/82ea49ff49fd4edcb552fd2907c88166.
2024/08/15 19:54:46 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/3.
2024/08/15 19:54:46 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 2 at: http://localhost:5000/#/experiments/3/runs/b47a536eb4f04d9fbe0d682436c363f7.
2024/08/15 19:54:46 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/3.
2024/08/15 19:54:49 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 3 at: http://localhost:5000/#/experiments/3/runs/770a3a3e2ab745d1b23d96d423ac713e.
2024/08/15 19:54:49 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/3.
2024/08/15 19:54:49 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 4 at: http://localhost:5000

Trial 10 achieved value: 0.45122011252601224 with  1.9892% improvement


2024/08/15 19:59:00 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 11 at: http://localhost:5000/#/experiments/3/runs/47abf8e26c934282a95db90728e8f56c.
2024/08/15 19:59:00 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/3.
2024/08/15 19:59:38 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 12 at: http://localhost:5000/#/experiments/3/runs/a442095ddbb9441b900dbde66116c23d.
2024/08/15 19:59:38 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/3.
2024/08/15 20:01:49 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 13 at: http://localhost:5000/#/experiments/3/runs/b1276842a80b4dafaf951980346950ec.
2024/08/15 20:01:49 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/3.


...

2024/08/15 20:57:50 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 78 at: http://localhost:5000/#/experiments/3/runs/8eac0bee9d9c492b99dd0c96c8910703.
2024/08/15 20:57:50 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/3.
2024/08/15 20:58:36 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 79 at: http://localhost:5000/#/experiments/3/runs/b77d8a399b9a4c7ab3bdc6688c02052f.
2024/08/15 20:58:36 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/3.


Trial 79 achieved value: 0.4473650852649761 with  0.2919% improvement


2024/08/15 20:59:15 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 80 at: http://localhost:5000/#/experiments/3/runs/24cf2b3fb191441fb46a6b0c5e64e819.
2024/08/15 20:59:15 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/3.
2024/08/15 21:00:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 81 at: http://localhost:5000/#/experiments/3/runs/670c2cf2b7574331b673ec55b88ad16a.
2024/08/15 21:00:13 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/3.
2024/08/15 21:01:17 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 82 at: http://localhost:5000/#/experiments/3/runs/dbaedd2a0bd048f0bc8be3b8171e4f61.
2024/08/15 21:01:17 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/3.
2024/08/15 21:02:34 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 83 at: http://localhost:

Trial 88 achieved value: 0.4469916211775324 with  0.0836% improvement


2024/08/15 21:05:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 89 at: http://localhost:5000/#/experiments/3/runs/dbd7b6a2874644e0af843301ee724957.
2024/08/15 21:05:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/3.
2024/08/15 21:05:39 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 90 at: http://localhost:5000/#/experiments/3/runs/76027043b5b941c1ab5c89ef5afafd70.
2024/08/15 21:05:39 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/3.
2024/08/15 21:06:29 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 91 at: http://localhost:5000/#/experiments/3/runs/57909cb9a4be446cb9301a8984a0ce43.
2024/08/15 21:06:29 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/3.
2024/08/15 21:07:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 92 at: http://localhost:

## Testeando el modelo

Una vez que el modelo fue entrenado, podemos levantarlo y testearlo de una forma agnóstica a donde está guardado.

In [9]:
loaded = mlflow.sklearn.load_model(model_uri)

In [10]:
import numpy as np

random_row = X_test.sample(n=1)

prediction = loaded.predict(np.array(random_row).reshape([1, -1]))[0]

reality = y_test.loc[random_row.index[0]]["cantidad_robos_log"]

print(f"Prediction: {prediction}")
print(f"Reality: {reality}")

Prediction: 2.172480344772339
Reality: 2.3978952727983707


## Registramos el modelo 

Realizamos el registro del modelo en MLflow. En este registro se pone el modelo productivo que luego se usará para servir en formato on-line.

In [11]:
from mlflow import MlflowClient

client = MlflowClient()
name = "crime_prediction_model_prod"
desc = "This predicts the number of crimes in the city of Buenos Aires given the weather conditions"

# Creamos el modelo productivo
client.create_registered_model(name=name, description=desc)

# Guardamos como tag los hiper-parametros en la version del modelo
tags = model.get_params()
tags["model"] = type(model).__name__
tags["mae"] = mae

# Guardamos la version del modelo
result = client.create_model_version(
    name=name,
    source=model_uri,
    run_id=model_uri.split("/")[-3],
    tags=tags
)

# Y creamos como la version con el alias de champion para poder levantarlo en nuestro
# proceso de servicio del modelo on-line.
client.set_registered_model_alias(name, "champion", result.version)

2024/08/15 21:14:05 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: crime_prediction_model_prod, version 1
